In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import sys
import os
import importlib.util

from google.colab import drive
drive.mount('/content/drive')

data_path_folder = "./drive/My Drive/Colab Notebooks/face_anti_spoofing/"
TRAIN_DIR = data_path_folder + 'train/'
TEST_DIR = data_path_folder + "test/"

n_real = len(os.listdir(TRAIN_DIR + "/real"))
n_spoof = len(os.listdir(TRAIN_DIR + "/spoof"))
n_real, n_spoof

# **Remove invalid files**

In [ ]:
def remove_img(directory: str):
    counter = 0
    for img_name in os.listdir(directory):
        if img_name.find('.') == 0:
            counter += 1
            os.remove(directory + img_name)
    print(f'removed {counter} files')

remove_img(TRAIN_DIR + "real/")
remove_img(TRAIN_DIR + "spoof/")
# remove_img(TEST_DIR)

# **Start**

In [ ]:
INPUT_SIZE = 224
BATCH_SIZE = 32

Augmnet data

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


val_share = 0.1

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=val_share,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale = 1./255,
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=1,
    class_mode='binary',
    shuffle=False
)

Plot samples

In [ ]:
import matplotlib.pyplot as plt
import math
%matplotlib inline


train_sample, y_train = next(train_generator)
val_sample, y_val = next(val_generator)
test_sample, y_test = next(test_generator)

fig, axes = plt.subplots(1, 3)

axes[0].imshow(train_sample[0])
axes[0].set_title('Train sample')

axes[1].imshow(val_sample[0])
axes[1].set_title('Val sample')

axes[2].imshow(test_sample[0])
axes[2].set_title('Test sample')

fig.set_figwidth(12)
fig.set_figheight(6)
plt.show()

In [ ]:
import tensorflow as tf

from keras import utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential, Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import ZeroPadding2D
from keras.layers import Activation
from keras.layers import GlobalAveragePooling2D
from keras.applications import MobileNetV2
from keras.models import load_model

from keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow_addons.callbacks import TQDMProgressBar

# **MobileNetV2**

In [ ]:
pretrain_MobileNetV2 = tf.keras.applications.MobileNetV2(
    input_shape=(INPUT_SIZE, INPUT_SIZE, 3),
    include_top=False,
    weights="imagenet",
)


# Add fully connected layer for classification
def add_fc_func_model(func_model):
    x = func_model.output
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = Dropout(0.2)(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=func_model.input, outputs=x)

    return model

In [ ]:
model = add_fc_func_model(pretrain_MobileNetV2)

opt = tf.keras.optimizers.Adam(learning_rate=0.00001)

class_weight = {0: (n_real + n_spoof) / n_real / 2, 1: (n_real + n_spoof) / n_spoof / 2}

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
with tf.device('/GPU:0'):
    model.fit_generator(
        train_generator,
        epochs=50,
        steps_per_epoch=(1 - val_share) * (n_real + n_spoof) // BATCH_SIZE,
        validation_data=val_generator,
        validation_steps=val_share * (n_real + n_spoof) // BATCH_SIZE,
        verbose=1,
        callbacks=[    
            ModelCheckpoint(
                data_path_folder + 'checkpoints/MobileNetV2' + 'weights.{epoch:02d}-{loss:.2f}.h5',
                monitor='val_loss',
                mode='min',
                save_best_only=True, 
                save_weights_only=False
            ),
        ],
        class_weight=class_weight
    )

# **Make predictions**

In [ ]:
import os

# model = load_model(data_path_folder + r"checkpointsMobileNetV2weights.01-0.21.h5")

filenames = test_generator.filenames
nb_samples = len(filenames)
with tf.device('/GPU:0'):    
    predict = model.predict_generator(test_generator,steps = nb_samples)

In [ ]:
len(predict), len(filenames)

Save to .txt and .csv

In [ ]:
with open(data_path_folder + 'predictions.txt', 'w', encoding='utf-8') as f:
    for file_name, prediction in zip(filenames, predict):
        f.write(file_name + ',' + '{0:.16f}'.format(1 - prediction[0]) + '\n')
f.close()

In [ ]:
with open(data_path_folder + 'predictions.txt', 'r') as f:
    lines = f.readlines()
new_lines = list(map(lambda line: line.split('/')[1], lines))

with open(data_path_folder + 'predictions_edited.txt', 'w') as f:
    f.write(''.join(new_lines))

In [ ]:
import csv


with open(data_path_folder + 'predictions_edited.txt', 'r') as infile, open(data_path_folder + 'predictions_table.csv', 'w') as outfile:
     stripped = (line.strip() for line in infile)
     lines = (line.split(",") for line in stripped if line)
     print(list(lines))
     writer = csv.writer(outfile, quoting=csv.QUOTE_NONNUMERIC)
     writer.writerows(lines)